In [1]:
import os
from typing import TypedDict
from dotenv import load_dotenv
from google import genai
from google.genai import types
from langgraph.graph import StateGraph, END

# --- Load .env variables ---
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError("Missing GEMINI_API_KEY in .env file")

# Initialize Gemini client
client = genai.Client(api_key=api_key)

In [2]:

# Wrapper for Gemini 2.5 Flash
def call_gemini(prompt: str, disable_thinking: bool = False) -> str:
    cfg = None
    if disable_thinking:
        cfg = types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0)
        )
    resp = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
        config=cfg,
    )
    return resp.text

# ---------------- Define State ----------------
class State(TypedDict, total=False):
    input: str
    summary: str
    translation: str

# ---------------- Nodes ----------------
def summarizer(state: State) -> State:
    prompt = "Summarize this:\n\n" + state["input"]
    return {"summary": call_gemini(prompt)}

def translator(state: State) -> State:
    prompt = "Translate this into Urdu:\n\n" + state["summary"]
    return {"translation": call_gemini(prompt)}

In [3]:

# ---------------- Build Graph ----------------
graph = StateGraph(State)
graph.add_node("summarizer", summarizer)
graph.add_node("translator", translator)
graph.set_entry_point("summarizer")
graph.add_edge("summarizer", "translator")
graph.add_edge("translator", END)
app = graph.compile()


In [4]:

# ---------------- Run ----------------
if __name__ == "__main__":
    input_text = "Mahad is studying in FAST university, haha less GPA."
    result = app.invoke({"input": input_text})

    print("INPUT:\n", result.get("input"))
    print("\nSUMMARY:\n", result.get("summary"))
    print("\nTRANSLATION (Urdu):\n", result.get("translation"))


INPUT:
 Mahad is studying in FAST university, haha less GPA.

SUMMARY:
 Mahad is a student at FAST university and has a low GPA.

TRANSLATION (Urdu):
 Here are a couple of ways to translate that into Urdu, both are natural:

**Option 1 (More common phrasing for "student at a university"):**
مہد فاسٹ یونیورسٹی کا طالب علم ہے اور اس کا جی پی اے کم ہے۔
*Mahad FAST University ka talib-e-ilm hai aur us ka GPA kam hai.*

**Option 2 (Slightly more literal for "at"):**
مہد فاسٹ یونیورسٹی میں ایک طالب علم ہے اور اس کا جی پی اے کم ہے۔
*Mahad FAST University mein aik talib-e-ilm hai aur us ka GPA kam hai.*

**Breakdown of terms:**
*   **Mahad:** مہد
*   **is a student:** طالب علم ہے (talib-e-ilm hai)
*   **at FAST university:** فاسٹ یونیورسٹی کا/میں (FAST University ka/mein)
*   **and:** اور (aur)
*   **has a low GPA:** اس کا جی پی اے کم ہے (us ka GPA kam hai - literally "his GPA is low")


In [5]:
print(app.get_graph().draw_mermaid())


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	summarizer(summarizer)
	translator(translator)
	__end__([<p>__end__</p>]):::last
	__start__ --> summarizer;
	summarizer --> translator;
	translator --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

